In [1]:
import findspark
findspark.add_packages("graphframes:graphframes:0.8.0-spark3.0-s_2.12")
findspark.init()
import pyspark
import os
os.environ['PYSPARK_SUBMIT_ARGS']="--master local[2] pyspark-shell"
sc = pyspark.SparkContext(appName = "MyAPP")
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
sc.addPyFile('/home/msbd5003/graphframes-0.8.0-spark3.0-s_2.12.jar')


/home/msbd5003/spark/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [2]:
import pandas as pd
import numpy as np
from datetime import date
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import *

In [3]:
off_train = spark.read.csv('ccf_offline_stage1_train.csv',header=True, inferSchema=True)
off_train.header = ['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received','Date']
off_test = spark.read.csv('ccf_offline_stage1_test_revised.csv', header=True, inferSchema=True)
off_test.header = ['User_id','Merchant_id','Coupon_id','Discount_rate','Distance','Date_received']
on_train = spark.read.csv('ccf_online_stage1_train.csv', header= True, inferSchema=True)
on_train.header = ['User_id','Merchant_id','Action','Coupon_id','Discount_rate','Date_received','Date']

In [4]:
dataset3 = off_test
feature3 = off_train.where(((off_train.Date>='20160315') & (off_train.Date<='20160630')) \
                                   | ((off_train.Date=='null' )& (off_train.Date_received>='20160315') & (off_train.Date_received<='20160630')))\
                              .select('*')
dataset2 =  off_train.where((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615'))\
                    .select('*')
feature2 = off_train.where((off_train.Date>='20160201')&(off_train.Date<='20160514')\
                           |((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514')))\
                    .select('*')
dataset1 = off_train.where((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514'))\
                    .select('*')
feature1 = off_train.where((off_train.Date>='20160101')&(off_train.Date<='20160413')\
                           |((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413')))\
                    .select('*')

In [5]:
print(feature3.count())
print(dataset2.count())
print(feature2.count())
print(dataset1.count())
print(feature1.count())

1036975
258446
812779
137167
995240


In [6]:

all_u_m = feature3.select('User_id','Merchant_id').distinct()
print(all_u_m.count())

#distinct = dropduplicates, can use keys seq[attributes]
#all_u_m2 = feature3.select('User_id','Merchant_id').dropDuplicates()
#print(all_u_m2.count())

491819


In [7]:
t = feature3.select('User_id','Merchant_id','Date')
t = t.select('User_id', 'Merchant_id')\
     .where(t.Date!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_total')

t1 = feature3.select('User_id','Merchant_id','Coupon_id')
t1 = t1.select('User_id', 'Merchant_id')\
     .where(t1.Coupon_id!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_received')

#t.show()
#t1.show()

t2 = feature3.select('User_id','Merchant_id','Date','Date_received')
t2 = t2.select('User_id', 'Merchant_id')\
     .where((t2.Date!='null') & (t2.Date_received != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_use_coupon')

t3 = feature3.select('User_id','Merchant_id','Date')
t3 = t3.select('User_id', 'Merchant_id')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_any')

t4 = feature3.select('User_id','Merchant_id','Date','Coupon_id')
t4 = t4.select('User_id', 'Merchant_id')\
     .where((t4.Date!='null')& (t4.Coupon_id != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_common')

In [8]:
user_merchant3 = all_u_m.join(t,['User_id','Merchant_id'])
user_merchant3 = user_merchant3.join(t1,['User_id','Merchant_id'])
user_merchant3 = user_merchant3.join(t2,['User_id','Merchant_id'])
user_merchant3 = user_merchant3.join(t3,['User_id','Merchant_id'])
user_merchant3 = user_merchant3.join(t4,['User_id','Merchant_id'])

In [10]:
null_re = udf(lambda x: 0 if(x == 'null') else x)

user_merchant3 = user_merchant3.withColumn('User_merchant_buy_use_coupon', null_re(col('user_merchant_buy_use_coupon')))
user_merchant3 = user_merchant3.withColumn('User_merchant_buy_common', null_re(col('user_merchant_buy_common')))

user_merchant3 = user_merchant3.withColumn('User_merchant_coupon_transfer_rate', lit(user_merchant3.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant3.User_merchant_received.cast(FloatType())))
user_merchant3 = user_merchant3.withColumn('User_merchant_coupon_buy_rate', lit(user_merchant3.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant3.User_merchant_buy_total.cast(FloatType())))
user_merchant3 = user_merchant3.withColumn('User_merchant_coupon_rate', lit(user_merchant3.User_merchant_buy_total\
                 .cast(FloatType()) / user_merchant3.User_merchant_any.cast(FloatType())))
user_merchant3 = user_merchant3.withColumn('User_merchant_coupon_common_rate', lit(user_merchant3.User_merchant_buy_common\
                 .cast(FloatType()) / user_merchant3.User_merchant_buy_total.cast(FloatType())))


user_merchant3.repartition(1).write.csv('user_merchant3.csv', header = True)

In [11]:
all_u_m = feature2.select('User_id','Merchant_id').distinct()
#print(all_u_m.count())

t = feature2.select('User_id','Merchant_id','Date')
t = t.select('User_id', 'Merchant_id')\
     .where(t.Date!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_total')

t1 = feature2.select('User_id','Merchant_id','Coupon_id')
t1 = t1.select('User_id', 'Merchant_id')\
     .where(t1.Coupon_id!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_received')

#t.show()
#t1.show()

t2 = feature2.select('User_id','Merchant_id','Date','Date_received')
t2 = t2.select('User_id', 'Merchant_id')\
     .where((t2.Date!='null')& (t2.Date_received != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_use_coupon')

t3 = feature2.select('User_id','Merchant_id','Date')
t3 = t3.select('User_id', 'Merchant_id')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_any')

t4 = feature2.select('User_id','Merchant_id','Date','Coupon_id')
t4 = t4.select('User_id', 'Merchant_id')\
     .where((t4.Date!='null')& (t4.Coupon_id != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_common')

In [12]:
user_merchant2 = all_u_m.join(t,['User_id','Merchant_id'])
user_merchant2 = user_merchant2.join(t1,['User_id','Merchant_id'])
user_merchant2 = user_merchant2.join(t2,['User_id','Merchant_id'])
user_merchant2 = user_merchant2.join(t3,['User_id','Merchant_id'])
user_merchant2 = user_merchant2.join(t4,['User_id','Merchant_id'])

In [13]:


user_merchant2 = user_merchant2.withColumn('User_merchant_buy_use_coupon', null_re(col('user_merchant_buy_use_coupon')))
user_merchant2 = user_merchant2.withColumn('User_merchant_buy_common', null_re(col('user_merchant_buy_common')))

user_merchant2 = user_merchant2.withColumn('User_merchant_coupon_transfer_rate', lit(user_merchant2.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant2.User_merchant_received.cast(FloatType())))
user_merchant2 = user_merchant2.withColumn('User_merchant_coupon_buy_rate', lit(user_merchant2.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant2.User_merchant_buy_total.cast(FloatType())))
user_merchant2 = user_merchant2.withColumn('User_merchant_coupon_rate', lit(user_merchant2.User_merchant_buy_total\
                 .cast(FloatType()) / user_merchant2.User_merchant_any.cast(FloatType())))
user_merchant2 = user_merchant2.withColumn('User_merchant_coupon_common_rate', lit(user_merchant2.User_merchant_buy_common\
                 .cast(FloatType()) / user_merchant2.User_merchant_buy_total.cast(FloatType())))

user_merchant2.repartition(1).write.csv('user_merchant2.csv', header = True)

In [14]:
all_u_m = feature1.select('User_id','Merchant_id').distinct()
#print(all_u_m.count())

t = feature1.select('User_id','Merchant_id','Date')
t = t.select('User_id', 'Merchant_id')\
     .where(t.Date!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_total')

t1 = feature1.select('User_id','Merchant_id','Coupon_id')
t1 = t1.select('User_id', 'Merchant_id')\
     .where(t1.Coupon_id!='null')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_received')

#t.show()
#t1.show()

t2 = feature1.select('User_id','Merchant_id','Date','Date_received')
t2 = t2.select('User_id', 'Merchant_id')\
     .where((t2.Date!='null')& (t2.Date_received != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_use_coupon')

t3 = feature1.select('User_id','Merchant_id','Date')
t3 = t3.select('User_id', 'Merchant_id')\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_any')

t4 = feature1.select('User_id','Merchant_id','Date','Coupon_id')
t4 = t4.select('User_id', 'Merchant_id')\
     .where((t4.Date!='null')& (t4.Coupon_id != 'null'))\
     .groupby('User_id', 'Merchant_id').count()\
     .withColumnRenamed('count', 'User_merchant_buy_common')

In [15]:
user_merchant1 = all_u_m.join(t,['User_id','Merchant_id'])
user_merchant1 = user_merchant1.join(t1,['User_id','Merchant_id'])
user_merchant1 = user_merchant1.join(t2,['User_id','Merchant_id'])
user_merchant1 = user_merchant1.join(t3,['User_id','Merchant_id'])
user_merchant1 = user_merchant1.join(t4,['User_id','Merchant_id'])

In [16]:


user_merchant1 = user_merchant1.withColumn('User_merchant_buy_use_coupon', null_re(col('user_merchant_buy_use_coupon')))
user_merchant1 = user_merchant1.withColumn('User_merchant_buy_common', null_re(col('user_merchant_buy_common')))

user_merchant1 = user_merchant1.withColumn('User_merchant_coupon_transfer_rate', lit(user_merchant1.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant1.User_merchant_received.cast(FloatType())))
user_merchant1 = user_merchant1.withColumn('User_merchant_coupon_buy_rate', lit(user_merchant1.User_merchant_buy_use_coupon\
                 .cast(FloatType()) / user_merchant1.User_merchant_buy_total.cast(FloatType())))
user_merchant1 = user_merchant1.withColumn('User_merchant_coupon_rate', lit(user_merchant1.User_merchant_buy_total\
                 .cast(FloatType()) / user_merchant1.User_merchant_any.cast(FloatType())))
user_merchant1 = user_merchant1.withColumn('User_merchant_coupon_common_rate', lit(user_merchant1.User_merchant_buy_common\
                 .cast(FloatType()) / user_merchant1.User_merchant_buy_total.cast(FloatType())))


user_merchant1.repartition(1).write.csv('user_merchant1.csv', header = True)